# Multi-Objective Optimization of Heat Exchangers

## Importing the modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import CoolProp.CoolProp as Cool
from scipy import integrate
from scipy import interpolate
from scipy.optimize import curve_fit
from scipy.optimize import fsolve

from pymoo.util.misc import stack
#from pymoo.model.problem import Problem

from pymoo.core.problem import Problem

#from pymoo.algorithms.nsga2 import NSGA2
from pymoo.algorithms.moo.nsga2 import NSGA2

from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination
from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover
from pymoo.optimize import minimize

#mpl.style.use('default')
#plt.rc('text',usetex = True)
#plt.rc('font', family='serif',size = 12)

## Calculating the Properties of the Fluid

In [ ]:
def Prop_Fluid(Temp, fluid ='Water'):
    
    """
    rho, cp, mu, Pr, k = Prop_Fluid(Temp,fluid)
    
    Function to calculate the density, specific heat, viscosity, Prandtl number and thermal conductivity of a determined fluid
    
    #Inputs:
    - Temp: Temperature of the fluid [ºC]
    - fluid: name of the fluid [-]
    
    # Outputs
    - rho: Density of the fluid [kg/m3] 
    - cp: Specific Heat of the fluid [kJ/kg.K]
    - mu: Viscosiity off the fluid [Pa.s]
    - Pr:  Prandtl numbber of the fluid [-] 
    - k: Thermal conductiviity of the fluid [W/(m2.K)]
    

    """
    
    rho = Cool.PropsSI('D','T',273.15 + Temp ,'P',101325, fluid)       # [kg/m3] Density
    cp  = Cool.PropsSI('C','T',273.15 + Temp ,'P',101325, fluid)       # [kJ/kg.K] Specific heat
    mu = Cool.PropsSI('V','T',273.15 + Temp ,'P',101325, fluid)        # [Pa.s] Viscosity
    Pr = Cool.PropsSI('PRANDTL','T',273.15 + Temp,'P',101325, fluid)   # [-] Prandtl Number
    k = Cool.PropsSI('L','T',273.15+Temp,'P',101325, fluid)            # [W/(m2.K)] Thermal Conductivity
    
    return rho, cp, mu,Pr, k

## Regression of the fan curve

In [ ]:
def fan(V,a,b,c):
    
    """
    Arbitrary funtion to do the fan regression
    """
    
    P = a*V**2+b*V + c
    return P

#### Fan Data 

In [4]:
dP_fan = np.array([0,20,40,60])
V_fan = np.array([3540,3245,2895,2480])/3600
eta_fan = (dP_fan*V_fan)/np.array([183,189,194,200])

Coef_fan = curve_fit(fan,V_fan,dP_fan,maxfev=10000)
Coef_eta = curve_fit(fan,V_fan,eta_fan,maxfev=10000)

## Input Parameters

In [ ]:
# Properties of the Air

rho_a, cp_a, mu_a,Pr_a, k_a = Prop_Fluid(28, 'Air')

#Properties of the metals

k_w = 273       # Thermal Conductivity of Aluminum
rho_al = 2700   # Density of Aluminum
rho_cu = 9000   # Density of Copper

## Calculation of the Characteristic Areas

In [ ]:
def HT_Area(Xl, Xt, De,sigmaf, theta, Beta, L1, L2, L3, N_t):
    
    """
    A_s, A_t, A_fr, A_min_o, sigma, D_h = HT_Area(Xl, Xt, De,sigmaf, theta, Beta, L1, L2, L3, N_t)
    
    Returns the main heat transfer area for the Heat Exchanger:
    
    Inputs:
    - Xl: Longitudinal Pitch [m] 
    - Xt: Transversal Pitch [m]
    - De: External tube diameter [m]
    - sigmaf: Thickness of the fin [m]
    - theta: fin inclination [-]
    - Beta: Fin density [1/m]
    - L1: Length of the Heat Exchanger [m]
    - L2: Width of the Heat Exchanger [m]
    - L3: Height of the Heat Exchanger Circuit [m]
    - N_t: Number of Tubes of the Heat exchanger Circuit [-]
    
    
    Outputs:
    - A_s: Total surface Area [m2]
    - A_t: External Tube Area [m2]
    - A_fr: External Frontal Area [m2]
    - A_f: Secondary heat transfer Area [m2]
    - A_min_o: External Minimum free flow area [m2]
    - sigma: External Minimum free-flow area divided by the frontal area [-]
    - D_h: External Hydraulic Diameter [m]
    """
    
    #Minimum Free Flow area
    
    a = ((Xt - De) - (Xt - De)*sigmaf/np.cos(theta)*Beta)/2
    b = ((Xt/2)**2+Xl**2)**0.5 - De - (Xt-De)*sigmaf/np.cos(theta)*Beta
    c = np.zeros(len(a))
    
    for i in range(len(a)):
        if a[i]<b[i]:
            c[i] = 2*a[i]
        else:
            c[i] = 2*b[i]
    A_min_o = ((L3/Xt-1)*c + (Xt-De)-(Xt-De)*sigmaf/np.cos(theta)*Beta)*L1 #Minimum free flow area
    
    
    
    #Surface Areas
    A_p = np.pi*De*(L1-sigmaf*Beta*L1/np.cos(theta))*N_t+2*(L2*L3 - np.pi*De**2*N_t/4) #Primary surface area  - exposed tubes and header plates
    A_f = 2*(L2*L3/np.cos(theta)-(np.pi*De**2/4)*N_t)*Beta*L1 + 2*L3*sigmaf*Beta*L1/np.cos(theta) # Secondary heat transfer area - Fins
    A_s = A_p + A_f       #Total surface area
    A_t = np.pi*De*L1*N_t #External tube area
    A_fr = L1*L3 #Frontal area
    sigma = A_min_o/A_fr
    D_h = 4*A_min_o*L2/A_s #Hydraulic diameter
    
    return A_s, A_t, A_fr, A_f, A_min_o, sigma, D_h

## Calculation of the External Pressure Drop

In [8]:
def Pressure_Drop(vazao, L1, L3, sigma, dc, pd, xf, pf, Xt, c1, A_s, A_t, D_h, N_r, c2, c3, A_min_o):
    
    """
    dP_o, dP_fan, Re, u_max = Pressure_Drop(vazao, L1, L3, sigma, dc, pd, xf, pf, Xt, c1, A_s, A_t, D_h, N_r, c2, c3, A_min_o)
    
    # Inputs:
    - vazao: Volumetric mass flow rate [m3/s]
    - L1: Length of the Heat Exchanger [m]
    - L3: Height of the Heat Exchanger [m]
    - sigma: External Minimum free-flow area divided by the frontal area
    - dc: Collar diameter [m]
    - pd: Fin pattern peak to valley distance [m]
    - xf: Fin project way length [m]
    - pf: Fin Pitch [m]
    - Xt: Transsversal tube pitch [mm]
    - c1: constant 1
    - A_s: Total surface Area [m2]
    - A_t: External Tube Areea [m2] 
    - D_h: Hydraulic Diameter [m]
    - N_r: Number of rows [-]
    - c2: constant 2
    - c3: constant 3
    - A_min_o: Minimum free-fow area
    
    # Outputs:
    
    - dP_o: External Pressurre Drop [Pa] 
    - dP_fan: Fan Head [Pa]
    - Re: Reynolds Number [-]
    - u_max: Maximum Velocity [m/s2]
    """
    
    
    u_air = vazao/(L1*L3)
    u_max = u_air/sigma
    Re = u_max*rho_a*dc/(mu_a)
    c4 = -10.2192/np.log(Re)
    f_o  = 0.05273*(Re**c2)*((pd/xf)**c3)*((pf/Xt)**c4)*((np.log(A_s/A_t))**(-2.726))*(D_h/dc)**0.1325*((N_r)**0.02305)   
    G_o = rho_a*u_max

    dP_o = G_o**2/(2*rho_a)*((f_o)*(A_s/A_min_o))
    dP_fan = fan(vazao,Coef_fan[0][0],Coef_fan[0][1],Coef_fan[0][2])
    
    return dP_o, dP_fan, Re, u_max

## Calculation of the Fin Efficciency

In [30]:
def fin_eff(h_o, sigmaf, Xl, Xt, Di, A_f, A_s):
    
    """
    eta_o = fin_eff(h_o, sigmaf, Xl, Xt, Di, A_f, A_s)
    
    #Inputs 
    - h_o: convective heat transfer coefficient [W/(m2.K)]
    - sigmaf: Thickness of the fin [m]
    - Xl: Longitudinal Pitch [m] 
    - Xt: Transversal Pitch [m]
    - Di: Internal Tube diameter [m]
    - A_f: Secondary heat transfer area [m2]
    - A_s: Total surface Area [m2]
    
    #Outpus:
    - eta_o: fin efficiency [-]
    """
    
    
    
    m = ((2*h_o)/(k_w*sigmaf))**0.5
    Pm = Xl/2
    Pl = (((Xt/2)**2+Xl**2)**0.5)/2
    R_eq = 1.27*Pm*(Pl/Pm-0.3)**0.5
    phi = (R_eq/(Di/2)-1)*(1+0.35*np.log(R_eq/(Di/2)))
    eta = np.tanh(m*(Di/2)*phi)/(m*(Di/2)*phi)
    eta_o = 1 - (A_f/A_s)*(1-eta)
    
    return eta_o

## Calculation of the Internal Flow Characteristics 

In [ ]:
def Internal_Flow(Temp, G_i, Di, Lh, N_t, Nucleos, K_bend, m_f, A_i):
    """
    dP_i, W_p, Rt_i = Internal_Flow(Temp, G_i, Di, Lh, N_t, Nucleos, K_bend, m_f, A_i):
    
    #Inputs:
    - Temp: Temperature of the fluid [ºC] 
    - G_i: Mass flux [kg/(m2.s)]
    - Di: Internal Tube diameter [m]
    - Lh: Corrected Length [m]
    - N_t: Number of Tubes [-]
    - Nucleos: Number of Circuits [-]
    - K_bend: Number of 180º Bends [-]
    - m_f: Mass flow rate [kg/s]
    - A_i: Internal Area of the Tube [m2]
    
    #Outputs:
    - dP_i: Internal Pressure Drop [Pa]
    - W_p: Pumping Power of the Internal Flow [W]
    - Rt_i: Thermal Resistance of the Internal Flow [W/K]
    """
    
    
    rho_w, cp_w, mu_w,Pr_w, k_w = Prop_Fluid(Temp, 'Water')
    Re_i = G_i*Di/mu_w 
    A = (2.457*np.log(1/((7/Re_i)**0.9+0.27*(0.0015*10**(-3)/Di))))**16
    B = (37530/Re_i)**16
    f = (8*((8/Re_i)**12 + (A + B)**(-3/2))**(1/12))/4
    dP_i = f*G_i**2*4*(Lh/Di + (N_t/Nucleos)*K_bend)/(2*rho_w)
    W_p = dP_i*m_f/rho_w
    Nu_i = np.zeros(len(Re_i))
    for i in range(len(Re_i)):
        if Re_i[i] < 2300:
            Nu_i[i] = 3.66
        else:
            Nu_i[i] = ((f[i]/2)*(Re_i[i]-1000)*Pr_w)/(1+12.7*((f[i]/2)**0.5)*(Pr_w**(2/3)-1)) 
    
    h_i = Nu_i*k_w/Di
    Rt_i = 1/(h_i*A_i)
    
    return  dP_i, W_p, Rt_i

## Calculation of the Effectivness of the HEx

In [ ]:
def Effectiv(Temp, Rt_o, R_tub, Rt_i, vazao, Nucleos, m_f, N_r):

    rho_w, cp_w, mu_w,Pr_w, k_w = Prop_Fluid(Temp, 'Water')
    
    UA = (Rt_o + R_tub + Rt_i)**-1
    NTU = UA/(vazao/Nucleos*rho_a*cp_a)
    C_r = (vazao/Nucleos*rho_a*cp_a)/(m_f*cp_w)
    
    
    Eff = np.zeros(len(C_r))
    for i in range(len(C_r)):
        #if N_r[i] == 1:
        #    Eff[i] = (1-np.exp(-C_r[i]*(1-np.exp(-NTU[i]))))/C_r[i]
        #elif N_r[i] == 2:
        #    K = (1-np.exp(-NTU[i]/2))
        #    Eff[i] = (1-np.exp(-2*K*C_r[i])*((1+C_r[i]*K**2)))/C_r[i]
        #elif N_r[i] == 3:
        #    K = (1-np.exp(-NTU[i]/3))
        #    Eff[i] = (1-np.exp(-3*K*C_r[i])*(1+C_r[i]*K**2*(3-K)+3*C_r[i]**2*K**4/2))/C_r[i]
        if N_r[i] ==4:
            K = (1-np.exp(-NTU[i]/4))
            Eff[i] = (1-np.exp(-4*K*C_r[i])*(1+C_r[i]*K**2*(6-4*K+K**2)+4*C_r[i]**2*K**4*(2-K)+8*C_r[i]**3*K**6/3))/C_r[i]
        else:
            Eff[i] = 1 - np.exp(NTU[i]**0.22*(np.exp(-C_r[i]*NTU[i]**0.78)-1)/C_r[i])
    
    ECr = Eff*C_r
    
    return ECr, Eff

## Calculation of the HEx Thermal-Hydraulic Performance

In [12]:
def HEx(HEx_Inputs):
    
    #Input parameters
    Xl,Xt,N_r,N_l,L1,Di,pf,Nucleos = HEx_Inputs

    # Fixed design parameters
    pd = 0.0018
    sigmah = 0.00030 
    sigmaf = 0.15*10**-3
    xf = 7*10**-3
    K_bend = 50
    
    # Preliminar calculations
    De = Di + 0.0008         # External Diamater
    m_f = m_f1/Nucleos       # Mass flow rate per Circuit
    Beta = 1/pf              # Fin Density
    Nfin = Beta*L1           # Number of fins
    dc = De + 2*sigmaf       # Collar Diameter
    theta = np.arctan(pd/xf) # Fin inclination
    L3 = (N_l*Xt)/2*Nucleos  # Height of the Hex Circuit
    L2 = N_r*Xl              # Width of the Hex
    N_t = N_r*N_l/2*Nucleos  # Number of Tubes per Circuit

    # Calculation of the areas 
    A_s, A_t, A_fr, A_f, A_min_o,sigma, D_h = HT_Area(Xl, Xt, De,sigmaf, theta, Beta, L1, L2, L3, N_t)
    
    
    #Pressure drop correlation parameters
    c1 = -0.1707 - 1.374*((Xl/sigmaf)**(-0.493))*((pf/dc)**(-0.886))*((N_r)**(-0.143))*((pd/xf)**(-0.0296))
    c2 = 0.1714 - 0.07372*((pf/Xl)**0.25)*(np.log(A_s/A_t))*((pd/xf)**(-0.2))
    c3 = 0.426*((pf/Xt)**0.3)*(np.log(A_s/A_t))    
    
    #Heat Exchanger-Fan Coupling
    def PressureDropDif(vazao):
        u_air = vazao/(L1*L3)
        u_max = u_air/sigma
        Re = u_max*rho_a*dc/(mu_a)
        c4 = -10.2192/np.log(Re)
        f_o  = 0.05273*(Re**c2)*((pd/xf)**c3)*((pf/Xt)**c4)*((np.log(A_s/A_t))**(-2.726))*(D_h/dc)**0.1325*((N_r)**0.02305)   
        G_o = rho_a*u_max

        dP_o = G_o**2/(2*rho_a)*((f_o)*(A_s/A_min_o))
        dP_fan = fan(vazao,Coef_fan[0][0],Coef_fan[0][1],Coef_fan[0][2])
        return(dP_o-dP_fan)
    
    vazao = fsolve(PressureDropDif,3000/3600*np.ones(len(Xl)))
    
    dP_o,dP_fan, Re, u_max = Pressure_Drop(vazao, L1, L3, sigma, dc, pd, xf, pf, Xt, c1, A_s, A_t, D_h, N_r, c2, c3, A_min_o)
    
    eta_fan = fan(vazao,Coef_eta[0][0],Coef_eta[0][1],Coef_eta[0][2])
    W_o = vazao*dP_o/eta_fan
    
    # Thermal coefficient calculations
    j_o = 1.7910*((Re)**c1)*((Xl/sigmaf)**(-0.456))*((N_r)**(-0.27))*((pf/dc)**(-1.343))*((pd/xf)**0.317)
    h_o = j_o*rho_a*u_max*cp_a*Pr_a**(-0.66666)

    #Fin efficiency
    eta_o = fin_eff(h_o, sigmaf, Xl, Xt, Di, A_f, A_s)
    
    #External Thermal Resistance
    Rt_o = 1/(eta_o*h_o*A_s/Nucleos)
    
    ## Internal Flow
    
    # Preliminar Calculations
    Lh=(L1+2*sigmah)*N_t/Nucleos  # Corrected Length
    A_i = np.pi*Di*L1*N_t/Nucleos # Internal Surface Area
    A_min_i = np.pi*(Di**2)/4     # Internal minimum free-flow area
    G_i = m_f/A_min_i             # Mass-flux of the internal flow
    
    # Pressure drop correlations:
    dP_i, W_p, Rt_i = Internal_Flow(22, G_i, Di, Lh, N_t, Nucleos, K_bend, m_f, A_i)
    #dP_i_h, W_p_h, Rt_i_h = Internal_Flow(35, G_i, Di, Lh, N_t, Nucleos, K_bend, m_f, A_i)
    
    #Thermal Resistance of the Tubes
    R_tub = np.log(De/Di)/(2*np.pi*L1*N_t/Nucleos*k_w)
    
    #Effectivness of the HEx
    ECr, Eff = Effectiv(22, Rt_o, R_tub, Rt_i, vazao, Nucleos, m_f, N_r)
    #ECr_h, Eff_h = Effectiv(35, Rt_o, R_tub, Rt_i, vazao, Nucleos, m_f, N_r):
    
    #Masses of the Components of the Heat exchanger
    m_fin = Beta*L1*(L2*L3/np.cos(theta)-np.pi*De**2*N_t)*sigmaf*rho_al
    m_hea = 2*(L2*L3-np.pi*De**2*N_t)*sigmah*rho_al
    m_tub = (np.pi*(De**2-Di**2)/4*(L1 + np.pi/2*(Xt**2+Xl**2)**0.5)*N_t)*rho_cu
    rho_w = 1000
    m_w   = (np.pi*Di**2/4*(L1+np.pi/2*(Xt**2+Xl**2)**0.5)*N_t)*rho_w

    return(ECr,W_p*Nucleos,W_o,m_fin,m_hea,m_tub,m_w) 

In [13]:
#Verificar ElementWise Problem

#https://pymoo.org/interface/problem.html

In [14]:
class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=8,
                         n_obj=2,
                         n_constr=0,
                         #xl=np.array([30*10**-3, 31.75*10**-3, 2, 2, 0.35, 0 , 3*10**-3, 3]),
                         #xu=np.array([33*10**-3  , 38.1*10**-3 , 4, 4, 0.55, 1 , 6*10**-3, 8]))
                         xl=np.array([30*10**-3, 31.75*10**-3, 4, 2, 0.35, 0 , 4 , 3]),
                         xu=np.array([33*10**-3, 38.1*10**-3 , 8, 4, 0.55, 1 , 9 , 8]))

    def _evaluate(self, x, out, *args, **kwargs):
        #Xl,Xt,N_r,N_l,L1,Di,pf,Nucleos
        Data_MOO = np.array([x[:,0],x[:,1],x[:,2],2*x[:,3],x[:,4],0.0117 + 0.0031*x[:,5],25.4/x[:,6]*10**-3,x[:,7]],dtype='float64')
        A,B,C,D,E,F,G = HEx(Data_MOO)
        
        
        f1 = -A
        f2 = B+C
        
        #g1 = C - 80
        #g2 = -B
        #g4 = 0.3 - D
        #g3 = D - 0.55

        out["F"] = np.column_stack([f1, f2])
        #out["G"] = np.column_stack([g1,g2,g3,g4])


problem = MyProblem()

In [22]:
termination = MultiObjectiveSpaceToleranceTermination(tol=0.01,
                                                      n_last=100,
                                                      nth_gen=10,
                                                      n_max_gen=None,
                                                      n_max_evals=None)

In [23]:
mask = ["real","real","int","int","real","int","int","int"]

In [24]:
sampling = MixedVariableSampling(mask, {
    "real": get_sampling("real_random"),
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random"),
    "int": get_sampling("int_random"),
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random"),
    "int": get_sampling("int_random"),
    "int": get_sampling("int_random")
    
})

crossover = MixedVariableCrossover(mask, {
    "real": get_crossover("real_sbx", prob=0.9, eta=15.0),
    "real": get_crossover("real_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "real": get_crossover("real_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0),
    "int": get_crossover("int_sbx", prob=0.9, eta=15.0)

})

mutation = MixedVariableMutation(mask, {
    "real": get_mutation("real_pm", eta=20.0),
    "real": get_mutation("real_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0),
    "real": get_mutation("real_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0),
    "int": get_mutation("int_pm", eta=20.0) ,
    "int": get_mutation("int_pm", eta=20.0)    
})

In [25]:
algorithm = NSGA2(
    pop_size=40,
    n_offsprings=500,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True
)

In [26]:
p_d = 0.0018
sigmah = 0.00030 
sigmaf = 0.15*10**-3
xf = 7*10**-3

In [27]:
def Opt():
    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=1,
                   pf=problem.pareto_front(use_cache=False),
                   save_history=False,
                   verbose=1)
    
    Data = np.array([(res.X.T[0]).tolist(),
        (res.X.T[1]).tolist(),
        (res.X.T[2]).tolist(),
        (2*res.X.T[3]).tolist(),
        (res.X.T[4]).tolist(),
        (0.0117 + 0.0031*res.X.T[5]).tolist(),
        (25.4/res.X.T[6]*10**-3).tolist(),
        xf*np.ones(len(res.X.T[0])),
        sigmaf*np.ones(len(res.X.T[0])),
        sigmah*np.ones(len(res.X.T[0])),
        p_d*np.ones(len(res.X.T[0])),
        m_f1*np.ones(len(res.X.T[0])),
        (res.X.T[7]).tolist()]).T
    return res.F.T,Data

In [28]:
mfr = np.arange(1800/3600,5450/3600,30/3600)
#mfr = [5000/360]

In [29]:
ECr_MOO = np.array([0])
W_MOO = np.array([0])
MFR = np.array([0])
Data = [np.array([0]*13),np.array([0]*13)]
for i in range(len(mfr)):
    print(mfr[i]*3600)
    m_f1 = mfr[i]
    Res, Data_MOO = Opt()
    ECr_MOO = np.hstack((ECr_MOO,-Res[0]))
    W_MOO = np.hstack((W_MOO,Res[1]))
    MFR = np.hstack((MFR,mfr[i]*np.ones(len(Res[0]))))
    Data = np.concatenate((Data,Data_MOO))

1800.0
n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |      40 |       5 |            - |            -
    2 |     540 |      11 |  0.172804516 |        ideal
    3 |    1040 |      32 |  0.923706443 |        ideal
    4 |    1540 |      40 |  0.039728506 |        ideal
    5 |    2040 |      40 |  0.171879818 |        ideal
    6 |    2540 |      40 |  0.210219190 |        ideal
    7 |    3040 |      40 |  0.005435144 |        ideal
    8 |    3540 |      40 |  0.585087161 |        ideal
    9 |    4040 |      40 |  0.004836088 |            f
   10 |    4540 |      40 |  0.010441278 |            f
   11 |    5040 |      40 |  0.005064250 |            f
   12 |    5540 |      40 |  0.004954116 |            f
   13 |    6040 |      40 |  0.004345460 |            f
   14 |    6540 |      40 |  0.007582816 |            f
   15 |    7040 |      40 |  0.012562573 |            f
   16 |    7540 |      40 |  0.006873854 |            f
   17 |    8040 |      40 |  0.004778266 

  145 |   72040 |      40 |  0.004082388 |            f
  146 |   72540 |      40 |  0.003442874 |            f
  147 |   73040 |      40 |  0.004750169 |            f
  148 |   73540 |      40 |  0.003522331 |            f
  149 |   74040 |      40 |  0.005439267 |            f
  150 |   74540 |      40 |  0.006130298 |            f
  151 |   75040 |      40 |  0.007313214 |            f
  152 |   75540 |      40 |  0.006053917 |            f
  153 |   76040 |      40 |  0.004751021 |            f
  154 |   76540 |      40 |  0.004049039 |            f
  155 |   77040 |      40 |  0.004079522 |            f
  156 |   77540 |      40 |  0.003843698 |            f
  157 |   78040 |      40 |  0.005701652 |            f
  158 |   78540 |      40 |  0.004927187 |            f
  159 |   79040 |      40 |  0.003239469 |            f
  160 |   79540 |      40 |  0.005302502 |            f
  161 |   80040 |      40 |  0.454571141 |        ideal
  162 |   80540 |      40 |  0.003452545 |      

   19 |    9040 |      40 |  0.009442164 |            f
   20 |    9540 |      40 |  0.024886348 |        ideal
   21 |   10040 |      40 |  0.006553029 |            f
   22 |   10540 |      40 |  0.003684326 |            f
   23 |   11040 |      40 |  0.007382277 |            f
   24 |   11540 |      40 |  0.008415615 |            f
   25 |   12040 |      40 |  0.004860411 |            f
   26 |   12540 |      40 |  0.005739800 |            f
   27 |   13040 |      40 |  0.006014196 |            f
   28 |   13540 |      40 |  0.003642165 |            f
   29 |   14040 |      40 |  0.006781009 |            f
   30 |   14540 |      40 |  0.004271468 |            f
   31 |   15040 |      40 |  0.010892092 |        ideal
   32 |   15540 |      40 |  0.004484439 |            f
   33 |   16040 |      40 |  0.004077704 |            f
   34 |   16540 |      40 |  0.004111393 |            f
   35 |   17040 |      40 |  0.004736901 |            f
   36 |   17540 |      40 |  0.005334592 |      

  166 |   82540 |      40 |  0.004713239 |            f
  167 |   83040 |      40 |  0.004383212 |            f
  168 |   83540 |      40 |  0.006330653 |            f
  169 |   84040 |      40 |  0.005427924 |            f
  170 |   84540 |      40 |  0.006155019 |            f
  171 |   85040 |      40 |  0.005068397 |            f
  172 |   85540 |      40 |  0.003902229 |            f
  173 |   86040 |      40 |  0.005236359 |            f
  174 |   86540 |      40 |  0.005907509 |            f
  175 |   87040 |      40 |  0.005223202 |            f
  176 |   87540 |      40 |  0.006241384 |            f
  177 |   88040 |      40 |  0.006602673 |            f
  178 |   88540 |      40 |  0.005987644 |            f
  179 |   89040 |      40 |  0.005038618 |            f
  180 |   89540 |      40 |  0.005394962 |            f
  181 |   90040 |      40 |  0.004610930 |            f
  182 |   90540 |      40 |  0.005621334 |            f
  183 |   91040 |      40 |  0.004300863 |      

  313 |  156040 |      40 |  0.003608123 |            f
  314 |  156540 |      40 |  0.007990135 |            f
  315 |  157040 |      40 |  0.004885369 |            f
  316 |  157540 |      40 |  0.006179046 |            f
  317 |  158040 |      40 |  0.004561923 |            f
  318 |  158540 |      40 |  0.005127871 |            f
  319 |  159040 |      40 |  0.005324599 |            f
  320 |  159540 |      40 |  0.004838420 |            f
  321 |  160040 |      40 |  0.007246126 |            f
  322 |  160540 |      40 |  0.004767339 |            f
  323 |  161040 |      40 |  0.005822846 |            f
  324 |  161540 |      40 |  0.004035370 |            f
  325 |  162040 |      40 |  0.008193705 |            f
  326 |  162540 |      40 |  0.007325494 |            f
  327 |  163040 |      40 |  0.005848066 |            f
  328 |  163540 |      40 |  0.024814314 |        ideal
  329 |  164040 |      40 |  0.005635284 |            f
  330 |  164540 |      40 |  0.003776465 |      

...

    5 |    2040 |      40 |  0.658479690 |        ideal
    6 |    2540 |      40 |  0.009771803 |        ideal
    7 |    3040 |      40 |  0.003412001 |        ideal
    8 |    3540 |      40 |  0.859849353 |        ideal
    9 |    4040 |      40 |  0.005871155 |            f
   10 |    4540 |      40 |  0.006031364 |            f
   11 |    5040 |      40 |  0.006243156 |            f
   12 |    5540 |      40 |  0.003936159 |            f
   13 |    6040 |      40 |  0.005656271 |            f
   14 |    6540 |      40 |  0.006339298 |            f
   15 |    7040 |      40 |  0.006013480 |            f
   16 |    7540 |      40 |  0.005260813 |            f
   17 |    8040 |      40 |  0.006697890 |            f
   18 |    8540 |      40 |  0.006616108 |            f
   19 |    9040 |      40 |  0.005962122 |            f
   20 |    9540 |      40 |  0.005330629 |            f
   21 |   10040 |      40 |  0.005334842 |            f
   22 |   10540 |      40 |  0.012867551 |      

  152 |   75540 |      40 |  0.005754499 |            f
  153 |   76040 |      40 |  0.004408232 |            f
  154 |   76540 |      40 |  0.005424215 |            f
  155 |   77040 |      40 |  0.006018194 |            f
  156 |   77540 |      40 |  0.005195742 |            f
  157 |   78040 |      40 |  0.004696235 |            f
  158 |   78540 |      40 |  0.005266794 |            f
  159 |   79040 |      40 |  0.007243008 |            f
  160 |   79540 |      40 |  0.005623505 |            f
  161 |   80040 |      40 |  0.006354313 |            f
  162 |   80540 |      40 |  0.004354849 |            f
  163 |   81040 |      40 |  0.006656651 |            f
  164 |   81540 |      40 |  0.008617184 |            f
  165 |   82040 |      40 |  0.005924122 |            f
  166 |   82540 |      40 |  0.007133257 |            f
  167 |   83040 |      40 |  0.005741938 |            f
  168 |   83540 |      40 |  0.003642508 |            f
  169 |   84040 |      40 |  0.004174083 |      

   75 |   37040 |      40 |  0.006901126 |            f
   76 |   37540 |      40 |  0.006563819 |            f
   77 |   38040 |      40 |  0.004653110 |            f
   78 |   38540 |      40 |  0.004869962 |            f
   79 |   39040 |      40 |  0.005877570 |            f
   80 |   39540 |      40 |  0.127563718 |        ideal
   81 |   40040 |      40 |  0.009964756 |            f
   82 |   40540 |      40 |  0.006734069 |            f
   83 |   41040 |      40 |  0.009792577 |            f
   84 |   41540 |      40 |  0.003887918 |            f
   85 |   42040 |      40 |  0.007225479 |            f
   86 |   42540 |      40 |  0.008040956 |            f
   87 |   43040 |      40 |  0.007827101 |            f
   88 |   43540 |      40 |  0.974243525 |        ideal
   89 |   44040 |      40 |  0.005560281 |            f
   90 |   44540 |      40 |  0.006982660 |            f
   91 |   45040 |      40 |  0.007545596 |            f
   92 |   45540 |      40 |  0.006127501 |      

  222 |  110540 |      40 |  0.005754449 |            f
  223 |  111040 |      40 |  0.005653063 |            f
  224 |  111540 |      40 |  0.004599105 |            f
  225 |  112040 |      40 |  0.007040064 |            f
  226 |  112540 |      40 |  0.005674805 |            f
  227 |  113040 |      40 |  0.006122247 |            f
  228 |  113540 |      40 |  0.005751914 |            f
  229 |  114040 |      40 |  0.005666145 |            f
  230 |  114540 |      40 |  0.005982174 |            f
  231 |  115040 |      40 |  0.004897840 |            f
  232 |  115540 |      40 |  0.006312480 |            f
  233 |  116040 |      40 |  0.005978507 |            f
  234 |  116540 |      40 |  0.005958203 |            f
  235 |  117040 |      40 |  0.003943995 |            f
  236 |  117540 |      40 |  0.006317129 |            f
  237 |  118040 |      40 |  0.004478075 |            f
  238 |  118540 |      40 |  0.008589750 |            f
  239 |  119040 |      40 |  0.006037258 |      

KeyboardInterrupt: 

In [ ]:
Nucleos = np.array([8])
Di = np.array([0.0117])
m_f = np.arange(4800/3600,5405/3600,50000/3600)
Xt = np.arange(31.75*10**-3,1.01*38.1*10**-3,(38.1*10**-3-31.75*10**-3)/4)
Xl = np.arange(33*10**-3,1.01*33.001*10**-3,(33*10**-3-27.5*10**-3)/4)
pf = np.array([25.4/8])*10**-3
N_r = np.arange(8,8.1,2)
N_l = np.array([4])
L1  = np.arange(0.55,0.56,0.1) #np.arange(0.4,0.73,0.05)
xf = 7*10**-3

In [ ]:
A = np.meshgrid(Xl,Xt,N_r,N_l,L1,Di,pf,xf,sigmaf,sigmah,p_d,m_f,Nucleos)
B = (np.array(A))
Data = B.T.reshape(-1,len(B))

Data = np.array([(res.X.T[0]).tolist(),
        (res.X.T[1]).tolist(),
        (2*res.X.T[2]).tolist(),
        (2*res.X.T[3]).tolist(),
        (res.X.T[4]).tolist(),
        (0.0117 + 0.0031*res.X.T[5]).tolist(),
        (res.X.T[6]).tolist(),
        xf*np.ones(len(res.X.T[0])),
        sigmaf*np.ones(len(res.X.T[0])),
        sigmah*np.ones(len(res.X.T[0])),
        p_d*np.ones(len(res.X.T[0])),
        m_f*np.ones(len(res.X.T[0])),
        (res.X.T[7]).tolist()]).T

In [ ]:
Res = (HEx(Data[0]))

In [ ]:
Res[10]

In [ ]:
#%%timeit

Res = np.zeros([len(Data),23])
Data_Final = np.zeros([len(Data),len(Data.T) + len(Res.T)])
for i in range (len(Data)):
    if  i % 1 == 0:
        print(i/(len(Data))*100)
    #if Data[i][1]*Data[i][3]*Data[i][12]/2>0.3 and Data[i][1]*Data[i][3]*Data[i][12]/2<0.55:
        Res[i] = (HEx(Data[i]))
        #if Res[i][10] < 80: #and Res[i][11] < 2548:
            #print(i)
        Data_Final[i] = np.concatenate([Data[i],Res[i]])     

In [ ]:
#m_f = mfr

In [ ]:
pd.DataFrame(Data_Final, columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])

In [ ]:
df = pd.DataFrame(Data_Final, columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])
df = df[df['m_f']>0]
df_mf = df.groupby('m_f')
df_mf.get_group(m_f[0])

In [ ]:
df.to_excel("Data_HEx_Tubal.xlsx")

In [ ]:
#df =pd.read_excel("Dissertation.xlsx", columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])

In [ ]:
Xl_1 = [[] for i in range(len(m_f))]
Xt_1 = [[] for i in range(len(m_f))]
Nr_1 = [[] for i in range(len(m_f))]
Nl_1 = [[] for i in range(len(m_f))]
L1_1 = [[] for i in range(len(m_f))]
Di_1 = [[] for i in range(len(m_f))]
pf_1 = [[] for i in range(len(m_f))]
mf_1 = [[] for i in range(len(m_f))]
Nucleos_1 = [[] for i in range(len(m_f))]
Ecrc_1 = [[] for i in range(len(m_f))]
Ecrh_1 = [[] for i in range(len(m_f))]
Efc_1 = [[] for i in range(len(m_f))]
Efh_1 = [[] for i in range(len(m_f))]
C_1 = [[] for i in range(len(m_f))]
NTUc_1 = [[] for i in range(len(m_f))]
NTUh_1 = [[] for i in range(len(m_f))]
Wpc_1 = [[] for i in range(len(m_f))]
Wph_1 = [[] for i in range(len(m_f))]
Wo_1 = [[] for i in range(len(m_f))]
dPo_1 = [[] for i in range(len(m_f))]
cfm_1 = [[] for i in range(len(m_f))]
Reo_1 = [[] for i in range(len(m_f))]
fo_1 = [[] for i in range(len(m_f))]
jo_1 = [[] for i in range(len(m_f))]
etao_1 = [[] for i in range(len(m_f))]
etaf_1 = [[] for i in range(len(m_f))]
mfin_1 = [[] for i in range(len(m_f))]
mhea_1 = [[] for i in range(len(m_f))]
mtub_1 = [[] for i in range(len(m_f))]
mwa_1 = [[] for i in range(len(m_f))]
dPc_1 = [[] for i in range(len(m_f))]
dPh_1 = [[] for i in range(len(m_f))]


ECr = [[] for i in range(len(m_f))]
W_Tc = [[] for i in range(len(m_f))]
W_Th = [[] for i in range(len(m_f))]
W_o = [[] for i in range(len(m_f))]
m_tot = [[] for i in range(len(m_f))]
Count = [[] for i in range(len(m_f))]

for i in range(len(m_f)):    
    ECr[i].append(np.array(df_mf.get_group(m_f[i])['ECr_c']))
    W_Tc[i].append(np.array(df_mf.get_group(m_f[i])['W_o']+df_mf.get_group(m_f[i])['W_p_c']))
    W_Th[i].append(np.array(df_mf.get_group(m_f[i])['W_o']+df_mf.get_group(m_f[i])['W_p_h']))
    m_tot[i].append(np.array(df_mf.get_group(m_f[i])['m_fin']+df_mf.get_group(m_f[i])['m_tub']+df_mf.get_group(m_f[i])['m_hea']+df_mf.get_group(m_f[i])['m_w']))
    W_o[i].append(np.array(df_mf.get_group(m_f[i])['W_p_c']))
    m_tot[i].append(np.array(df_mf.get_group(m_f[i])['m_fin']+df_mf.get_group(m_f[i])['m_tub']+df_mf.get_group(m_f[i])['m_hea']+df_mf.get_group(m_f[i])['m_w']))
    
    Xl_1[i].append(np.array(df_mf.get_group(m_f[i])['Xl']))
    Xt_1[i].append(np.array(df_mf.get_group(m_f[i])['Xt']))
    Nr_1[i].append(np.array(df_mf.get_group(m_f[i])['N_r']))
    Nl_1[i].append(np.array(df_mf.get_group(m_f[i])['N_l']))
    L1_1[i].append(np.array(df_mf.get_group(m_f[i])['L1']))
    Di_1[i].append(np.array(df_mf.get_group(m_f[i])['Di']))
    pf_1[i].append(np.array(df_mf.get_group(m_f[i])['pf']))
    mf_1[i].append(np.array(df_mf.get_group(m_f[i])['m_f']))
    Nucleos_1[i].append(np.array(df_mf.get_group(m_f[i])['Nucleos']))
    Ecrc_1[i].append(np.array(df_mf.get_group(m_f[i])['ECr_c']))
    Ecrh_1[i].append(np.array(df_mf.get_group(m_f[i])['ECr_h']))
    Efc_1[i].append(np.array(df_mf.get_group(m_f[i])['Eff_c']))
    Efh_1[i].append(np.array(df_mf.get_group(m_f[i])['Eff_h']))
    C_1[i].append(np.array(df_mf.get_group(m_f[i])['C_r']))
    NTUc_1[i].append(np.array(df_mf.get_group(m_f[i])['NTU_c']))
    NTUh_1[i].append(np.array(df_mf.get_group(m_f[i])['NTU_h']))
    Wpc_1[i].append(np.array(df_mf.get_group(m_f[i])['W_p_c']))
    Wph_1[i].append(np.array(df_mf.get_group(m_f[i])['W_p_h']))
    Wo_1[i].append(np.array(df_mf.get_group(m_f[i])['W_o']))
    dPo_1[i].append(np.array(df_mf.get_group(m_f[i])['dP_o']))
    cfm_1[i].append(np.array(df_mf.get_group(m_f[i])['vazao*3600']))
    Reo_1[i].append(np.array(df_mf.get_group(m_f[i])['Re_o']))
    fo_1[i].append(np.array(df_mf.get_group(m_f[i])['f_o']))
    jo_1[i].append(np.array(df_mf.get_group(m_f[i])['j_o']))
    etao_1[i].append(np.array(df_mf.get_group(m_f[i])['eta_o']))
    etaf_1[i].append(np.array(df_mf.get_group(m_f[i])['eta_fan']))
    mfin_1[i].append(np.array(df_mf.get_group(m_f[i])['m_fin']))
    mhea_1[i].append(np.array(df_mf.get_group(m_f[i])['m_hea']))
    mtub_1[i].append(np.array(df_mf.get_group(m_f[i])['m_tub']))
    mwa_1[i].append(np.array(df_mf.get_group(m_f[i])['m_w']))
    dPc_1[i].append(np.array(df_mf.get_group(m_f[i])['dP_c']))
    dPh_1[i].append(np.array(df_mf.get_group(m_f[i])['dP_h']))
    

In [ ]:
Cont = [[] for i in range(len(m_f))]
Ec_p_L = [[] for i in range(len(m_f))]
W_T_pc =  [[] for i in range(len(m_f))]
cfm_p_L = [[] for i in range(len(m_f))]

for k in range(np.size(m_f)):
    A = np.zeros(len(ECr[k][0]))
    for i in range(len(ECr[k][0])):
        for j in range(len(ECr[k][0])):
            #if ECr[k][0][i] < ECr[k][0][j] and cfm_1[k][0][i] > cfm_1[k][0][j]:
            if ECr[k][0][i] < ECr[k][0][j] and W_Tc[k][0][i] > W_Tc[k][0][j]:
                A[i] = 1
                break
        if A[i] == 0:
            Cont[k].append(i)
            Ec_p_L[k].append(ECr[k][0][i])
            W_T_pc[k].append(W_Tc[k][0][i])
            #cfm_p_L[k].append(cfm_1[k][0][i])
            #print(Cont[k],Ec_p_L[k],W_p_L[k])
    print(k/len(m_f)*100)

In [ ]:
Xl_p = []
Xt_p = []
Nr_p = []
Nl_p = []
L1_p = []
Di_p = []
pf_p = []
mf_p = []
Nucleos_p = []
Ecrc_p = []
Ecrh_p = []
Efc_p = []
Efh_p = []
C_p = []
NTUc_p = []
NTUh_p = []
Wpc_p = []
Wph_p = []
Wo_p = []
dPo_p = []
cfm_p = []
Reo_p = []
fo_p = []
jo_p = []
etao_p = []
etaf_p = []
mfin_p = []
mhea_p = []
mtub_p = []
mwa_p = []
dPc_p = []
dPh_p = []
    
    
for k in range(np.size(m_f)):
    for i in range(len(Cont[k])):
        Xl_p.append(Xl_1[k][0][Cont[k][i]])
        Xt_p.append(Xt_1[k][0][Cont[k][i]])
        Nr_p.append(Nr_1[k][0][Cont[k][i]])
        Nl_p.append(Nl_1[k][0][Cont[k][i]])
        L1_p.append(L1_1[k][0][Cont[k][i]])
        Di_p.append(Di_1[k][0][Cont[k][i]])
        pf_p.append(pf_1[k][0][Cont[k][i]])
        mf_p.append(mf_1[k][0][Cont[k][i]])
        Nucleos_p.append(Nucleos_1[k][0][Cont[k][i]])
        Ecrc_p.append(Ecrc_1[k][0][Cont[k][i]])
        Ecrh_p.append(Ecrh_1[k][0][Cont[k][i]])
        Efc_p.append(Efc_1[k][0][Cont[k][i]])
        Efh_p.append(Efh_1[k][0][Cont[k][i]])
        C_p.append(C_1[k][0][Cont[k][i]])
        NTUc_p.append(NTUc_1[k][0][Cont[k][i]])
        NTUh_p.append(NTUh_1[k][0][Cont[k][i]])
        Wpc_p.append(Wpc_1[k][0][Cont[k][i]])
        Wph_p.append(Wph_1[k][0][Cont[k][i]])
        Wo_p.append(Wo_1[k][0][Cont[k][i]])
        dPo_p.append(dPo_1[k][0][Cont[k][i]])
        cfm_p.append(cfm_1[k][0][Cont[k][i]])
        Reo_p.append(Reo_1[k][0][Cont[k][i]])
        fo_p.append(fo_1[k][0][Cont[k][i]])
        jo_p.append(jo_1[k][0][Cont[k][i]])
        etao_p.append(etao_1[k][0][Cont[k][i]])
        etaf_p.append(etaf_1[k][0][Cont[k][i]])
        mfin_p.append(mfin_1[k][0][Cont[k][i]])
        mhea_p.append(mhea_1[k][0][Cont[k][i]])
        mtub_p.append(mtub_1[k][0][Cont[k][i]])
        mwa_p.append(mwa_1[k][0][Cont[k][i]])
        dPc_p.append(dPc_1[k][0][Cont[k][i]])
        dPh_p.append(dPh_1[k][0][Cont[k][i]])

In [ ]:
plt.plot(mf_p,np.array(cfm_p)*0.588,'x')

In [ ]:
#plt.style.use('seaborn-whitegrid')
plt.style.use('default')
plt.rc('font',family = 'serif',size = 12)
plt.rc('text', usetex = True)

In [ ]:
plt.plot(Reo_p,fo_p,'x',color = '#1f77b4')
plt.plot(Reo_p,jo_p,'x',color = '#d62728')

plt.grid(linestyle = 'dashed')

In [ ]:
Data_Final = pd.DataFrame({'Xl':Xl_p,
                'Xt':Xt_p,
                'Nr':Nr_p,
                'Nl':Nl_p, 
                'L1':L1_p, 
                'Di':Di_p, 
                'pf':pf_p,
                'mf':mf_p, 
                'Nucleos':Nucleos_p, 
                'ECrC':Ecrc_p, 
                'ECrH':Ecrh_p, 
                'EffC':Efc_p, 
                'EffH':Efh_p, 
                'Cr':C_p, 
                'NTUC':NTUc_p, 
                'NTUH':NTUh_p, 
                'Wpc':Wpc_p, 
                'Wph':Wph_p, 
                'Wo':Wo_p, 
                'dPo':dPo_p,
                'vazao':cfm_p,
                'Reo':Reo_p,
                'fo':fo_p,
                'jo':jo_p,
                'etao':etao_p,
                'etaf':etaf_p,
                'mfin':mfin_p,
                'mhea':mhea_p,
                'mtub':mtub_p,
                'mwa':mwa_p,
                'dPc':dPc_p,
                'dPh':dPh_p,
             })

In [ ]:
Data_Final.to_excel("HEx_APEn_2.xlsx") 

In [ ]:
Data_Final

In [ ]:
plt.plot(np.array(Ecrc_p[0:len(Ec_p_L[0])]),np.array(pf_p[0:len(Ec_p_L[0])])*1000,'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 4800 kg/h')
plt.plot(np.array(Ecrc_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),np.array(pf_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])])*1000,'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 6400 kg/h')
plt.plot(np.array(Ecrc_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),np.array(pf_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])])*1000,'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 8000 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$p_\mathrm{fin}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#sc.set_alpha(0.75)
plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0,0.16])

plt.savefig('HEx_pf.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(np.array(Ec_p_L[0:len(Ec_p_L[0])]),Ec_p_L[0],'o',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 2500 kg/h')
plt.plot(np.array(Ec_p_L[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),Ec_p_L[1],'x',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 5000 kg/h')
plt.plot(np.array(Ec_p_L[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),Ec_p_L[2],'P',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 7500 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{V}\llap{---}_\mathrm{a}$ [cfm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#sc.set_alpha(0.5)
plt.legend(bbox_to_anchor=[0.55, 0.8])

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
plt.ylim([0.03,0.23])

#plt.savefig('HEx_Paretto.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
np.shape(np.array(W_T_pc)[0])

In [ ]:
plt.plot(np.array(W_T_pc)[0],np.array(Ec_p_L)[0],'o',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(res.F.T[1],-res.F.T[0],'o')
plt.plot(W_MOO,ECr_MOO,'x')
#plt.plot(np.array(W_T_pc)[1],np.array(Ec_p_L)[1],'x',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(np.array(W_T_pc)[2],np.array(Ec_p_L)[2],'P',alpha = 1, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#sc.set_alpha(0.5)
plt.legend(bbox_to_anchor=[0.55, 0.8])
#plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.03,0.23])

#plt.savefig('HEx_Paretto.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')
plt.savefig('HEx_Paretto.jpg',format = 'jpg',dpi = 300,bbox_inches = 'tight')

In [ ]:
H_p = np.array(Xt_p)*np.array(Nl_p)*np.array(Nucleos_p)/2
L_p = np.array(Xl_p)*np.array(Nr_p)

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(L_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(L_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(L_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$L_\mathrm{HEx}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.28, 0.43])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.29,0.57])

#plt.savefig('HEx_L.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(H_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(H_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(H_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$H_\mathrm{HEx}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.25, 0.17])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([0.06,0.27])

plt.savefig('HEx_H.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(L1_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(L1_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(L1_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$W_\mathrm{HEx}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.25, 0.33])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([0.06,0.30])

plt.savefig('HEx_W.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(Di_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
plt.plot(Ec_p_L[1],1000*np.array(Di_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(Di_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$D_\mathrm{i}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
#plt.legend(bbox_to_anchor=[0.25, 0.33])
plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.29,0.57])

#plt.savefig('HEx_Di.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
plt.plot(Ec_p_L[0],1000*np.array(pf_p[0:len(Ec_p_L[0])]),'o',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 2400 kg/h')
#plt.plot(Ec_p_L[1],1000*np.array(pf_p[len(Ec_p_L[0]):len(Ec_p_L[0])+len(Ec_p_L[1])]),'x',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 3900 kg/h')
#plt.plot(Ec_p_L[2],1000*np.array(pf_p[len(Ec_p_L[0])+len(Ec_p_L[1]):len(Ec_p_L[0])+len(Ec_p_L[1])+len(Ec_p_L[2])]),'P',alpha = 0.75, label = '$\dot{m}_\mathrm{HEx,l}$ = 5400 kg/h')
#plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]),'x')
#plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]),'s')

plt.xlabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.ylabel('$p_\mathrm{fin}$ [mm]')
plt.grid(linestyle = 'dashed')
#plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
#plt.legend(*sc.legend_elements("sizes", num=6), title = 'm$_\mathrm{HEx}$ [Kg]',bbox_to_anchor=[1.55, 0.45])
#plt.set_alpha(0.75)
plt.legend(bbox_to_anchor=[0.5, 0.4])
#plt.legend()
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0.29,0.57])

#plt.savefig('HEx_pf.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:

plt.plot(Ec_p_L[0],(Xl_p[0:len(Ec_p_L[0])]-min(Xl_p[0:len(Ec_p_L[0])]))/(max(Xl_p[0:len(Ec_p_L[0])])-min(Xl_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Xt_p[0:len(Ec_p_L[0])]-min(Xt_p[0:len(Ec_p_L[0])]))/(max(Xt_p[0:len(Ec_p_L[0])])-min(Xt_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(L1_p[0:len(Ec_p_L[0])]-min(L1_p[0:len(Ec_p_L[0])]))/(max(L1_p[0:len(Ec_p_L[0])])-min(L1_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Nucleos_p[0:len(Ec_p_L[0])]-min(Nucleos_p[0:len(Ec_p_L[0])]))/(max(Nucleos_p[0:len(Ec_p_L[0])])-min(Nucleos_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Nr_p[0:len(Ec_p_L[0])]-min(Nr_p[0:len(Ec_p_L[0])]))/(max(Nr_p[0:len(Ec_p_L[0])])-min(Nr_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(Nl_p[0:len(Ec_p_L[0])]-min(Nl_p[0:len(Ec_p_L[0])]))/(max(Nl_p[0:len(Ec_p_L[0])])-min(Nl_p[0:len(Ec_p_L[0])])),'o')
plt.plot(Ec_p_L[0],(pf_p[0:len(Ec_p_L[0])]-min(pf_p[0:len(Ec_p_L[0])]))/(max(pf_p[0:len(Ec_p_L[0])])-min(pf_p[0:len(Ec_p_L[0])])),'o')


Data_Final.to_excel("Results.xlsx") 

In [ ]:
EcrC = Data_Final['ECrC']
#WTT = Data_Final['Wo'] + Data_Final['Wpc']
WTT =  Data_Final['Wpc']

In [ ]:
DataF = Data_Final.groupby('mf')
Ecr_mf = DataF.get_group(m_f[0])['ECrC']
WTT_mf = DataF.get_group(m_f[0])['Wo'] + DataF.get_group(m_f[0])['Wpc']

In [ ]:
len(WTT_mf)

In [ ]:
import matplotlib.style
import matplotlib as mpl
mpl.style.use('default')
plt.rc('text',usetex = True)
plt.rc('font', family='serif',size = 12)
cmap = ('viridis')

In [ ]:
plt.grid(linestyle = 'dashed')
plt.scatter(WTT,EcrC, alpha = 0.5, c = Data_Final['mf'],cmap = 'viridis')

#plt.scatter(WTT_mf,Ecr_mf)
#lt.plot(WTT,Ecr,'o')
plt.colorbar(label = 'Mass flow rate [kg/h]')
plt.ylabel(r'$\varepsilon C_\mathrm{r}$ [-]')
plt.xlabel('Power [W]')


In [ ]:
Wt_1 = WTT
Ec_1 = EcrC

In [ ]:
plt.semilogx(Wt_1,Ec_1,'o',label = 'N$_\mathrm{Circ}$ = 1')
plt.semilogx(Wt_2,Ec_2,'x',label = 'N$_\mathrm{Circ}$ = 2')
plt.semilogx(Wt_3,Ec_3,'s',label = 'N$_\mathrm{Circ}$ = 3')
plt.semilogx(Wt_4,Ec_4,'^',label = 'N$_\mathrm{Circ}$ = 4')
plt.semilogx(Wt_6,Ec_6,'v',label = 'N$_\mathrm{Circ}$ = 6')
#plt.semilogx(Wt_12,Ec_12,'.',label = 'N$_\mathrm{Circ}$ = 12')
plt.legend()
plt.grid(linestyle = 'dashed')
plt.xlim(0,10000)
plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx,pump}$ [W]')
plt.savefig('N_Circ.pdf',format = 'pdf', dpi=1000,bbox_inches='tight')

In [ ]:
s1 = 'L1'
size = (5*(np.array(df_mf.get_group(m_f[0])[s1]) - min(np.array(df_mf.get_group(m_f[0])[s1])))/(max(np.array(df_mf.get_group(m_f[0])[s1])) - min(np.array(df_mf.get_group(m_f[0])[s1]))))**2



c1 = 'mf'
#col = np.array(df_mf.get_group(m_f[0])[c1])
size =  m_tot[0][0]
col = np.array(df_mf.get_group(m_f[0])[s1])

In [ ]:
DataF = Data_Final.groupby('mf')
Ecr = DataF.get_group(m_f[0])['ECrC']
WTT = DataF.get_group(m_f[0])['Wo'] + DataF.get_group(m_f[0])['Wpc']

In [ ]:
Data_Final['W_total'] = Data_Final['Wo'] + Data_Final['Wpc']

In [ ]:
Data_Final[Data_Final['W_total']<350]

In [ ]:
DataF.get_group(m_f[0])

In [ ]:
DataFi = Data_Final
DataF = DataFi.groupby('mf')

In [ ]:
#DataF = Data_Final.groupby('mf')
col  = np.array(DataF.get_group(m_f[0])['L1'])
size = np.array(DataF.get_group(m_f[0])['mfin']+DataF.get_group(m_f[0])['mhea']+DataF.get_group(m_f[0])['mwa']+DataF.get_group(m_f[0])['mtub'])
Ecr_mf = DataF.get_group(m_f[0])['ECrC']
WTT_mf = DataF.get_group(m_f[0])['Wo'] + DataF.get_group(m_f[0])['Wpc']

m_tot_1 = mfin_1+mhea_1+mtub_1+mwa_1


In [ ]:
DFF = pd.DataFrame(np.array([cfm_1[2][0],ECr[2][0],W_Tc[2][0],m_tot[2][0]]).T,
                              columns = ['cfm','ECr','W','m'])

In [ ]:
DFF['W']

In [ ]:
DFF= DFF[DFF['W']<350]


In [ ]:
i = 0
sc = plt.scatter(DFF['cfm'],DFF['ECr'],marker = 'o',cmap = 'jet', c = DFF['W'],s = DFF['m'],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')
#sc = plt.scatter(cfm_1[i][0]*0.588,ECr[i][0],marker = 'o',cmap = 'jet', c= Xl_1[i][0])#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

plt.plot(np.array(cfm_p_L[i]),Ec_p_L[i],'X',color = 'black', label = 'Pareto Front')
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{W}_\mathrm{HEx}$ [W]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]',bbox_to_anchor=[1.55, 0.45])
sc.set_alpha(0.5)
#plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.ylim([0,0.16])

#plt.savefig('HEx_5400_cfm.pdf',format = 'pdf',bbox_inches='tight')
plt.savefig('HEx_5400_cfm.jpg',format = 'jpg',dpi = 300,bbox_inches = 'tight')

In [ ]:
i = 0

sc = plt.scatter(W_Tc[i][0],ECr[i][0],marker = 'o',cmap = 'jet', c = cfm_1[i][0],s = m_tot[i][0],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

#plt.plot(WTT,Ecr,'x',color = 'black', mfc='none', label = 'Pareto Front',alpha = 0.65)
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx}$ [W]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]')
sc.set_alpha(0.5)
#plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([146,170.5])
plt.savefig('HEx_2400.pdf',format = 'pdf',bbox_inches='tight')

In [ ]:
i = 1

sc = plt.scatter(W_Tc[i][0],ECr[i][0],marker = 'o',cmap = 'jet', c = cfm_1[i][0],s = m_tot[i][0],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

#plt.plot(WTT,Ecr,'x',color = 'black', mfc='none', label = 'Pareto Front',alpha = 0.65)
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx}$ [W]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]')
sc.set_alpha(0.5)
#plt.legend()

#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([146,170.5])
plt.savefig('HEx_3900.pdf',format = 'pdf',bbox_inches='tight')

In [ ]:
i = 0

sc = plt.scatter(W_Tc[i][0],ECr[i][0],marker = 'o',cmap = 'jet', c = cfm_1[i][0],s = m_tot[i][0],rasterized=True)#, alpha = 1, s = size, c = col,cmap = 'viridis')

Ecr = DataF.get_group(m_f[i])['ECrC']
WTT = DataF.get_group(m_f[i])['Wo'] + DataF.get_group(m_f[i])['Wpc']

#plt.plot(WTT,Ecr,'x',color = 'black', mfc='none', label = 'Pareto Front',alpha = 0.65)
#plt.set_legend()
#plt.plot(np.array(W_p_L[0]),np.array(Ec_p_L[0]),'o',alpha = 0.8)

plt.ylabel(r'$(\varepsilon C^\mathrm{*})_\mathrm{HEx}$ [-]')
plt.xlabel('$\dot{W}_\mathrm{HEx}$ [W]')
plt.grid(linestyle = 'dashed')
plt.colorbar(label = '$\dot{V}\llap{---}_\mathrm{a}$ [m$^3$/h]')
plt.legend(*sc.legend_elements("sizes", num=6), title = '$m_\mathrm{HEx}$ [kg]')
sc.set_alpha(0.5)
#plt.legend()
#plt.xlim([0,350])
#plt.savefig(HEx, dpi=None, facecolor='w', edgecolor='w',
        #orientation='portrait', papertype=None, format=None,
        #transparent=False, bbox_inches=None, pad_inches=0.1,
        #frameon=None, metadata=None)      
#plt.xlim([146,170.5])
plt.savefig('HEx_5400.pdf',format = 'pdf',bbox_inches='tight')

In [ ]:
#plt.scatter(m_tot[0][0],ECr[0][0], c = np.array(df_mf.get_group(m_f[0])['m_w']/m_tot[0][0]))
plt.scatter(m_tot[0][0],ECr[0][0], c = col)
plt.colorbar(label = 'Internal Pumping Power [W]')
plt.grid()

In [ ]:
df_final = pd.DataFrame(columns = ['Xl','Xt','N_r','N_l','L1','Di','pf','xf','sigmaf','sigmah','p_d','m_f','Nucleos','ECr_c','ECr_h','Eff_c','Eff_h','C_r','NTU_c','NTU_h','W_p_c','W_p_h','W_o','dP_o','vazao*3600','Re_o','f_o','j_o','eta_o','eta_fan','m_fin','m_hea','m_tub','m_w','dP_c','dP_h'])


In [ ]:
A = [[] for i in range(len(Cont[0]))]

In [ ]:
np.array(df)[Cont[0][0]]

In [ ]:
A[1].append(np.array(df)[Cont[0][1]])

In [ ]:
A = np.array(df)[Cont[0][1]]

In [ ]:
A

In [ ]:
A.append(np.array(df)[Cont[0][0]])

In [ ]:
df_final.append(np.array(df)[Cont[0][0]])